# Test prompting Open AI

In [ ]:
import pandas as pd
import openai
import time
import os
from tqdm import tqdm

# Get OpenAI API KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("Missing OPENAI_API_KEY in environment variables.")
-

# Instancie un client OpenAI
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Paramètres
batch_size = 2
sleep_time = 1  # Temps de repos entre les batchs
max_retries = 3  # Nombre d'essais API en cas d'échec
valid_responses = ["oui","non"]

df = pd.read_csv("../data/processed/simplification.csv")
df = df.dropna(subset=['EXPOSE_SOMMAIRE']).reset_index(drop=True)

df = df.head(6)

start_index = 0

print(f"Traitement à partir de l'index {start_index}")

Traitement à partir de l'index 0


In [ ]:

# Prompt d'instruction
instruction_system = (
    "Tu es expert en analyse politique et en compréhension des mécanismes d'influence législative. "
    "Tu dois analyser des amendements par rapport à l'idée suivante : "
    '"limiter les délais de recours administratifs : accélérer les projets agricoles en réduisant les freins juridiques perçus comme des obstacles, '
    'en particulier les normes environnementales." '
    "Pour chaque amendement donné, réponds uniquement par : 'en faveur', 'neutre' ou 'opposé'. "
    "Ne donne aucune justification, seulement le terme demandé."
)

DESCRIPTION = "Simplifier les démarches administratives, démarches bureaucratiques chronophages qui sont des freins qui entravent le quotidien des agriculteurs"
IDEE="simplification des démarches administratives"
EXPOSE_SOMMAIRE = "Dans l’optique de renforcer l’efficacité du guichet unique et l’accompagnement des cédants et afin de rendre incontournable le dispositif France Services Agriculture pour tous les acteurs, il est proposé que l’attestation de passage à FSA constitue une pièce obligatoire du dossier de demande de retraite. "

instruction_system =(
     "Tu es un assistant expert en analyse sémantique et en inférence."
     "Ta tâche est de déterminer si un exposé sommaire d’amendement est en lien, même de manière implicite ou indirecte, avec une idée politique ou conceptuelle."
     "Tu dois suivre ces étapes:"
     "1. Lire l’exposé sommaire"
     "2. Identifier les thèmes, préoccupations ou objectifs présents, même s'ils ne sont pas formulés explicitement."
     "3. Lire la description de l’idée et résumer les concepts clés qu’elle porte."
     "4. Comparer les deux et indiquer si l’idée est en lien (même implicite) avec l’exposé."
     "5. Réponds uniquement à la fin par \"Oui\" ou \"Non\"."
     ""
     "Idée : "
     "{IDEE}"
     ""
     "Description de l’idée :"
     "{DESCRIPTION}"
     "---"
     ""
     ""
     "Si l'idée décrite est en lien indirect ou direct avec le contenu de l'exposé sommaire: Réponds uniquement par 'oui'"
     "Sinon répond 'non'"
     "Ne donne aucune justification, seulement le terme demandé."
)

# Fonction d'appel API avec retry automatique
def batch_query(batch_texts):
    messages = [{"role": "system", "content": instruction_system}]
    for idx, amendement in enumerate(batch_texts):
        messages.append({
            "role": "user",
            "content": f"Amendement {idx+1} : {amendement}"
        })
    
    for attempt in range(max_retries):
        try:
            print(messages)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=0,
                n=2
            )
            content = response.choices[0].message.content.strip().lower()

            print("Réponse brute GPT :")
            print(content)

            # ✅ Nouveau découpage
            split_results = []
            for line in content.split("\n"):
                if ":" in line:
                    _, response_text = line.split(":", 1)
                    split_results.append(response_text.strip())

            # Validation
            results = []
                        # Compléter si besoin
            print(results)
            for res in split_results:
                res_clean = res.strip().lower()
                if res_clean in valid_responses:
                    results.append(res_clean)
                else:
                    results.append("réponse invalide")

            # Compléter si besoin
            print(split_results)
            while len(results) < len(batch_texts):
                results.append("réponse manquante")
            
            return results
        
        except Exception as e:
            print(f"Erreur API tentative {attempt+1}/{max_retries} : {e}")
            time.sleep((attempt + 1) * 5)
    return ["échec API"] * len(batch_texts)


In [ ]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": instruction_system}
       {"role": "user", "Amendement 1": {EXPOSE_SOMMAIRE}}
    ]
)
print(response.choices[0].message.content)


Je suis désolé, il semble qu'il y ait une erreur. Je ne peux pas analyser sans avoir les informations de l'idée, de la description de l'idée et de l'exposé sommaire. Pourriez-vous s'il vous plaît me donner ces informations ?


In [25]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": instruction_system}
    ]
)

print(response.choices[0].message["content"])

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


# No batch

In [ ]:
response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": instruction_system}],
                temperature=0,
                n=2
            )

In [23]:

# Boucle principale avec barre de progression
for i in tqdm(range(start_index, len(df), batch_size), desc="Traitement des amendements"):
    batch = df['EXPOSE_SOMMAIRE'][i:i+batch_size].tolist()
    ids = df.index[i:i+batch_size].tolist()

    results = batch_query(batch)

    batch_df = pd.DataFrame({
        'Index': ids,
        'EXPOSE_SOMMAIRE': batch,
        'ResultatAnalyse': results
    })
    
    print(batch_df)
    # result_df = pd.concat([result_df, batch_df], ignore_index=True)
    
    # Sauvegarde régulière
    result_df.to_csv(output_file, index=False)
    
    time.sleep(sleep_time)

print("✅ Analyse terminée.")

Traitement des amendements:   0%|          | 0/3 [00:00<?, ?it/s]

[{'role': 'system', 'content': 'Tu es un assistant expert en analyse sémantique et en inférence.Ta tâche est de déterminer si un exposé sommaire d’amendement est en lien, même de manière implicite ou indirecte, avec une idée politique ou conceptuelle.Tu dois suivre ces étapes :1. Lire l’exposé sommaire et en donner un bref résumé.2. Identifier les thèmes, préoccupations ou objectifs présents, même s\'ils ne sont pas formulés explicitement.3. Lire la description de l’idée et résumer les concepts clés qu’elle porte.4. Comparer les deux et indiquer si l’idée est en lien (même implicite) avec l’exposé.5. Réponds uniquement à la fin par "Oui" ou "Non".Idée : {IDEE}Description de l’idée :{DESCRIPTION}---Analyse:1. Résumé de l’exposé :2. Thèmes identifiés :3. Concepts de l’idée :4. Comparaison sémantique :5. L’exposé est-il en lien (même implicite) avec cette idée ?Si l\'idée décrite est en lien indirect ou direct avec le contenu de l\'exposé sommaire: Réponds uniquement par \'oui\'Sinon répo

Traitement des amendements:   0%|          | 0/3 [00:02<?, ?it/s]

Réponse brute GPT :
idee : protection de l'environnement et promotion des pratiques durables.
description : l'idée vise à encourager des pratiques respectueuses de l'environnement, telles que la protection des berges et la promotion des haies, pour assurer la durabilité des ressources naturelles.

---

analyse:
1. amendement 1 : propose de rendre obligatoire l'attestation de passage à france services agriculture pour la demande de retraite des acteurs agricoles, en soulignant l'importance de l'accompagnement et de l'efficacité du guichet unique.
2. thèmes identifiés : accompagnement des acteurs agricoles, efficacité des services, consensus régional et national, objectifs d'intérêt général de la politique agricole.
3. concepts de l’idée : protection de l'environnement, pratiques durables, promotion des haies, simplification des règlementations.
4. comparaison sémantique : l'amendement 1 ne traite pas directement de la protection de l'environnement ou de la promotion des pratiques durabl

ValueError: All arrays must be of the same length